# Препроцессинг
**Задачи:**
* ✔ имплементировать правила замены букв и буквосочетаний
* ✔ скомпилировать словари матерных, грубых и ругательных слов
* имплементировать замену по расстоянию Левенштейна и словарю
* ✔ собрать оба подхода в двухступенчатый алгоритм

Ссылки на использованные словари:  
https://gist.github.com/nestyme/8531fe4ec34cd2c8e9b306513cb8b59a (Zueva et al.) 89 слов  
https://github.com/bohdan1/AbusiveLanguageDataset/blob/master/bad_words.txt (Andrusyak et al.) 623 слова  
Из первого были удалены слова, не относящиеся к ругательным, из второго - повторы, имеющиеся в первом

### Преколы и вопросы:
* в словаре Андрусяка некоторые слова даны в разных сиклонениях и в разных в т.ч. искажённых написаниях. Удалить для чистоты эксперимента?
* что делать с леммами
* в словаре пайморфи некоторые искажённые формы существуют (*кули*, *мля*). Возможно, есть смысл не делать проверку на существование и выполнять замены везде

### Замена букв по правилам
1. ✔ Взять слово, не содержащее пунктуацию
2. ✔ Проверить, существует ли слово
3. ✔ Если нет, то заменить по очереди все пары букв и буквосочетаний по очереди  
    (именно буквосочетаний и по очереди, потому что в слове *ипать* заменить надо *ип-еб*, а не *и-е*, *п-б*, *а-о* по очереди или разом)  
4. ✔ Сравнить, если ли в словаре ругательств полученный результат. Если да, заменить на него
5. ✔ Если результат не нашёлся, проверить все возможные комбинации одновременных замен
6. ✔ Если не нашлось и так, то вернуть исходное слово


### Расстояние Левенштейна до словаря мата
1. ✔ Взять слово, содержащее некириллические символы
2. ✔  Посчитать регуляркой их количество n
3. Найти расстояния Левенштейна m1, ..., mn до каждого из слов в словаре ругательств
4. Если n == m, то это корректная форма искомого слова  
Если нет n == m, то брать слово с минимальным m. На случай, если пропущено несколько букв или одна буква заменена несколькими символами  
5. Заменить слово на это


#### Правила замены:
йо → ё  
^мл → бл  
ип → еб  
п → б  
к → х  
т → д  
а → о  
с → з  
и → е  

к → г  
ш → ж  
ф → в  
3.14 → пи  
3,14 → пи

In [265]:
import re
import json
import enchant
import itertools
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [174]:
test_string = 'ипать йобаный ибать епать гавно гавном мля кули 3.14дор'
bad_wordlist = ['ебать', 'ёбаный', 'ебаный', 'говно', 'бля', 'хули', 'пидор']

In [108]:
# потом можно удалить из кода и оставить только в файле
replace_dict = {'йо': 'ё', 
                'ип': 'еб',
                'мл': 'бл',
                'ау': 'ов',
                'и': 'е',
                'п': 'б',
                'т': 'д',
                'к': 'х',
                'а': 'о',
                'с': 'з',
                'ш': 'ж',
                'ф': 'в',
                'у': 'в',
                '3.14': 'пи',
                '3,14': 'пи'}
with open('replacement.json', 'w') as f:
    json.dump(replace_dict, f)

In [109]:
with open('replacement.json', 'r') as f:
    replace_dict = json.load(f)

#### Собирание замены букв по правилам

In [311]:
nonletter_pat = re.compile('[^а-яёА-ЯЁ]')
pi_pat = re.compile('^3[.,]14.+')

def contains_nonletters(word):
    '''
    returns True, if given word contains any character that is not a cyrillic letter or
    "3.14" / "3,14" sequence
    '''
    if bool(re.search(nonletter_pat, word)) and not bool(re.search(pi_pat, word)):
        return True
    return False

In [312]:
def word_exists(word):
    '''
    checks whether given word is in OpenCorpora dictionary using PyMorphy2
    '''
    if morph.word_is_known(word):
        # return True
        return False
    return False

In [313]:
def correct_by_letters(word):
    '''
    takes a word, replaces letters one pair at a time unless the result or its lemma is
    found in the bad dictionary, otherwise returns the intial word
    '''
    for old, new in replace_dict.items():
        if old in word:
            new_word = word.replace(old, new)
            if morph.parse(new_word)[0].normal_form in bad_wordlist or new_word in bad_wordlist:
                return new_word
    # if the word is not found, go through all possible combinations of rules
    for l in range(1, len(replace_dict)):
        for tple in itertools.combinations(replace_dict.keys(), l+1):
            new_word = word
            for key in tple:
                new_word = new_word.replace(key, replace_dict[key])
        if morph.parse(new_word)[0].normal_form in bad_wordlist or new_word in bad_wordlist:
            return new_word
    # if still not found, return the initial input
    return word

#### Собирание Левенштейна

In [324]:
nonletter_pat = re.compile('[^а-яёА-ЯЁ]')

def count_nonletters(word):
    return len(re.findall(nonletter_pat, word))

In [327]:
print(count_nonletters('с00ба)ка)'))
print(count_nonletters('б***'))
print(count_nonletters('пи3дец'))
print(count_nonletters('собака'))

4
3
1
0


### Финальная функция:

In [314]:
def preprocess_distortion(text, debug=False):
    '''
    converts text to lowercase and performs all steps of checks and corrections for each token
    '''
    new_text = text.lower().split()
    
    for i, token in enumerate(new_text):
        if not contains_nonletters(token): # skip token if it has non-cyrillic characters
            if not word_exists(token): # skip token if it is an existing word
                if debug:
                    print(correct_by_letters(token))
                new_text[i] = correct_by_letters(token)

        else: # it has non-cyrillic characters and therefore is passed to Levenstein
            #new_text[i] = correct_by_levenstein(token) 
            pass
        
    return ' '.join(new_text)

In [315]:
test_string = 'ипать йобаный ибать епать гавно гавном мля кули 3.14дор 3,14тар алаоааллдлдлдв собака'
print(test_string)
preprocess_distortion(test_string)

ипать йобаный ибать епать гавно гавном мля кули 3.14дор 3,14тар алаоааллдлдлдв собака


'ебать ёбаный ебать ебать говно говном бля хули пидор пидор алаоааллдлдлдв собака'

In [316]:
## туду: расстояние левенштейна